In [36]:
import pandas as pd
import numpy as np

In [26]:
file_path = r"C:\Users\Poovesh Kumar M\Downloads\Planning\B1 katchon\0_Data\katchon_Amazon_com_weekly_data.csv"

# Read the CSV file into a DataFrame
df_base = pd.read_csv(file_path)

df_base = df_base[df_base['Week_Start_date'] <= '2024-08-26']

# Display the DataFrame
df_base.head()

,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,inventory_available
0,NaN,B00CH1LB3A,2022-03-28,2022-04-03,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,0.0,0
1,NaN,B00CH1LB3A,2022-04-04,2022-04-10,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,0.0,0
2,NaN,B00CH1LB3A,2022-04-11,2022-04-17,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,0.0,0
3,NaN,B00CH1LB3A,2022-04-18,2022-04-24,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,0.0,0
4,NaN,B00CH1LB3A,2022-04-25,2022-05-01,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,0.0,0


In [27]:
# print(len(df_base))
print("Asin count in base data: ", df_base.asin.nunique())
df_base_with_gmv_gt_zero = df_base[df_base['gmv_units']>0]
print("Asin count where gmv_units > 0: ", df_base_with_gmv_gt_zero.asin.nunique())

Asin count in base data:  5847
Asin count where gmv_units > 0:  4834


In [28]:
sales_week_df = df_base_with_gmv_gt_zero.groupby('asin').agg({ 'Week_Start_date':['nunique','min', 'max'],'gmv_units':['mean',np.std]}).reset_index()
sales_week_df.columns = [' '.join(col).strip() for col in sales_week_df.columns.values]
sales_week_df

,asin,Week_Start_date nunique,Week_Start_date min,Week_Start_date max,gmv_units mean,gmv_units std
0,B01G07XAWQ,55,2022-05-16,2024-05-20,26.345455,44.025758
1,B01H30M0DY,59,2022-05-16,2023-07-10,19.796610,32.025733
2,B01MPZZJI1,5,2022-05-16,2022-09-19,3.200000,3.346640
3,B01MQTMZR0,53,2022-05-16,2023-11-13,10.226415,17.636982
4,B01N8YTZ8R,1,2022-06-27,2022-06-27,1.000000,NaN
...,...,...,...,...,...,...
4829,B0D8481965,4,2024-08-05,2024-08-26,6.000000,4.966555
4830,B0D8LC7ZDV,4,2024-08-05,2024-08-26,10.250000,12.606215
4831,B0D9432PXK,2,2024-08-19,2024-08-26,5.000000,5.656854
4832,B0D9M93CR4,1,2024-08-26,2024-08-26,5.000000,NaN


In [29]:
sales_week_df['Week_Start_date max'] = pd.to_datetime(sales_week_df['Week_Start_date max'], format='%Y-%m-%d')
sales_week_df['Week_Start_date min'] = pd.to_datetime(sales_week_df['Week_Start_date min'], format='%Y-%m-%d')
sales_week_df['n_weeks'] = ((sales_week_df['Week_Start_date max'] - sales_week_df['Week_Start_date min']).dt.days)/7 
# sales_week_df
sales_week_df['ADI'] =  sales_week_df['n_weeks'] / (sales_week_df['Week_Start_date nunique'])

# eg: B0D7HZR4XL - Some edge case here ADI going > 1

sales_week_df['CV_2'] = (sales_week_df['gmv_units std']/sales_week_df['gmv_units mean'])**2

sales_week_df


,asin,Week_Start_date nunique,Week_Start_date min,Week_Start_date max,gmv_units mean,gmv_units std,n_weeks,ADI,CV_2
0,B01G07XAWQ,55,2022-05-16,2024-05-20,26.345455,44.025758,105.0,1.909091,2.792559
1,B01H30M0DY,59,2022-05-16,2023-07-10,19.796610,32.025733,60.0,1.016949,2.617077
2,B01MPZZJI1,5,2022-05-16,2022-09-19,3.200000,3.346640,18.0,3.600000,1.093750
3,B01MQTMZR0,53,2022-05-16,2023-11-13,10.226415,17.636982,78.0,1.471698,2.974416
4,B01N8YTZ8R,1,2022-06-27,2022-06-27,1.000000,NaN,0.0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
4829,B0D8481965,4,2024-08-05,2024-08-26,6.000000,4.966555,3.0,0.750000,0.685185
4830,B0D8LC7ZDV,4,2024-08-05,2024-08-26,10.250000,12.606215,3.0,0.750000,1.512592
4831,B0D9432PXK,2,2024-08-19,2024-08-26,5.000000,5.656854,1.0,0.500000,1.280000
4832,B0D9M93CR4,1,2024-08-26,2024-08-26,5.000000,NaN,0.0,0.000000,NaN


In [30]:
def classification(x):
    if x['ADI']<1.32 and x['CV_2']<0.49:
        return 'Smooth'
    elif x['ADI']>=1.32 and x['CV_2']<0.49:
        return 'Intermittent'
    elif x['ADI']<1.32 and x['CV_2']>=0.49:
        return 'Erratic'
    elif x['ADI']>=1.32 and x['CV_2']>=0.49:
        return 'Lumpy'
    else:
        return 'NA'

In [31]:
sales_week_df['classification'] = sales_week_df.apply(lambda x: classification(x), axis=1)
sales_week_df

,asin,Week_Start_date nunique,Week_Start_date min,Week_Start_date max,gmv_units mean,gmv_units std,n_weeks,ADI,CV_2,classification
0,B01G07XAWQ,55,2022-05-16,2024-05-20,26.345455,44.025758,105.0,1.909091,2.792559,Lumpy
1,B01H30M0DY,59,2022-05-16,2023-07-10,19.796610,32.025733,60.0,1.016949,2.617077,Erratic
2,B01MPZZJI1,5,2022-05-16,2022-09-19,3.200000,3.346640,18.0,3.600000,1.093750,Lumpy
3,B01MQTMZR0,53,2022-05-16,2023-11-13,10.226415,17.636982,78.0,1.471698,2.974416,Lumpy
4,B01N8YTZ8R,1,2022-06-27,2022-06-27,1.000000,NaN,0.0,0.000000,NaN,NA
...,...,...,...,...,...,...,...,...,...,...
4829,B0D8481965,4,2024-08-05,2024-08-26,6.000000,4.966555,3.0,0.750000,0.685185,Erratic
4830,B0D8LC7ZDV,4,2024-08-05,2024-08-26,10.250000,12.606215,3.0,0.750000,1.512592,Erratic
4831,B0D9432PXK,2,2024-08-19,2024-08-26,5.000000,5.656854,1.0,0.500000,1.280000,Erratic
4832,B0D9M93CR4,1,2024-08-26,2024-08-26,5.000000,NaN,0.0,0.000000,NaN,NA


In [32]:
category_analysis_df = sales_week_df.groupby('classification').agg({ 'asin':'nunique'}).reset_index()
category_analysis_df

,classification,asin
0,Erratic,1335
1,Intermittent,266
2,Lumpy,2074
3,NA,64
4,Smooth,1095


In [35]:
sales_week_df.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B1 katchon/1 US/KA_US_ASIN_classification_weekly.csv', index=False)
